# Partie 2 : Extraction des caractéristiques en utilisant Transformer ViT

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.applications.resnet import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os


### 1.4 Extraction en utilisant un transformer ViT

In [ ]:
from transformers import AutoFeatureExtractor, AutoModel
from PIL import Image
import numpy as np
import os

# Charger le modèle et le préprocesseur
model_name = "google/vit-base-patch16-224"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
transformer_model = AutoModel.from_pretrained(model_name)

def extract_transformer_features(image_path, feature_extractor, model):
    """
    Extrait les caractéristiques à partir d'une image en utilisant un modèle Transformer.

    Args:
        image_path (str): Chemin de l'image.
        feature_extractor (AutoFeatureExtractor): Préprocesseur du modèle Transformer.
        model (AutoModel): Modèle Transformer préentraîné.

    Returns:
        np.array: Vecteur de caractéristiques.
    """
    # Charger et prétraiter l'image
    image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt")

    # Passer dans le modèle
    outputs = model(**inputs)
    # Utiliser la sortie du dernier token CLS pour les caractéristiques globales
    features = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return features.flatten()

def process_transformer_dataset(folder_path, feature_extractor, model):
    features, labels = [], []

    # Calculate the total number of images
    total_images = sum(len(os.listdir(os.path.join(folder_path, label_folder))) for label_folder in ["O", "R"])

    processed_images = 0
    for label_folder, label in [("O", 0), ("R", 1)]:
        full_path = os.path.join(folder_path, label_folder)
        for image_name in os.listdir(full_path):
            image_path = os.path.join(full_path, image_name)
            feature = extract_transformer_features(image_path, feature_extractor, model)
            features.append(feature)
            labels.append(label)

            # Update processed images and print the progress
            processed_images += 1
            progress = (processed_images / total_images) * 100
            print(f"Image {processed_images}/{total_images} processed - {progress:.2f}% completed")

    return np.array(features), np.array(labels)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Exemple d'utilisation avec ViT
###Traiin
train_folder = "/content/g_dechets/g_dechets/Train"  # Mettez ici le chemin vers votre dossier principal
X_train_transformer, y_train_transformer = process_transformer_dataset(train_folder, feature_extractor, transformer_model)

print(f"Caractéristiques Transformer extraites(train) : {X_train_transformer.shape}")


Streaming output truncated to the last 5000 lines.
Image 6818/11816 processed - 57.70% completed
Image 6819/11816 processed - 57.71% completed
Image 6820/11816 processed - 57.72% completed
Image 6821/11816 processed - 57.73% completed
Image 6822/11816 processed - 57.74% completed
Image 6823/11816 processed - 57.74% completed
Image 6824/11816 processed - 57.75% completed
Image 6825/11816 processed - 57.76% completed
Image 6826/11816 processed - 57.77% completed
Image 6827/11816 processed - 57.78% completed
Image 6828/11816 processed - 57.79% completed
Image 6829/11816 processed - 57.79% completed
Image 6830/11816 processed - 57.80% completed
Image 6831/11816 processed - 57.81% completed
Image 6832/11816 processed - 57.82% completed
Image 6833/11816 processed - 57.83% completed
Image 6834/11816 processed - 57.84% completed
Image 6835/11816 processed - 57.85% completed
Image 6836/11816 processed - 57.85% completed
Image 6837/11816 processed - 57.86% completed
Image 6838/11816 processed - 

In [ ]:
import pandas as pd

##train transformer
transformer_df = pd.DataFrame(X_train_transformer)
transformer_df.to_csv('features_cnn_transformer(train).csv', index=False)

labels_trans_train = pd.DataFrame(y_train_transformer, columns=['Label'])
labels_trans_train.to_csv("y_train_trans.csv", index=False)

In [ ]:

###Test
test_folder = "/content/g_dechets/g_dechets/Test"  # Mettez ici le chemin vers votre dossier principal
X_test_transformer, y_test_transformer = process_transformer_dataset(test_folder, feature_extractor, transformer_model)

print(f"Caractéristiques Transformer extraites(test) : {X_test_transformer.shape}")

Streaming output truncated to the last 5000 lines.
Image 66/5064 processed - 1.30% completed
Image 67/5064 processed - 1.32% completed
Image 68/5064 processed - 1.34% completed
Image 69/5064 processed - 1.36% completed
Image 70/5064 processed - 1.38% completed
Image 71/5064 processed - 1.40% completed
Image 72/5064 processed - 1.42% completed
Image 73/5064 processed - 1.44% completed
Image 74/5064 processed - 1.46% completed
Image 75/5064 processed - 1.48% completed
Image 76/5064 processed - 1.50% completed
Image 77/5064 processed - 1.52% completed
Image 78/5064 processed - 1.54% completed
Image 79/5064 processed - 1.56% completed
Image 80/5064 processed - 1.58% completed
Image 81/5064 processed - 1.60% completed
Image 82/5064 processed - 1.62% completed
Image 83/5064 processed - 1.64% completed
Image 84/5064 processed - 1.66% completed
Image 85/5064 processed - 1.68% completed
Image 86/5064 processed - 1.70% completed
Image 87/5064 processed - 1.72% completed
Image 88/5064 processed -

In [ ]:
##test transformer
transformer1_df = pd.DataFrame(X_test_transformer)
transformer1_df.to_csv('features_cnn_transformer(test).csv', index=False)

labels_trans_test = pd.DataFrame(y_test_transformer, columns=['Label'])
labels_trans_test.to_csv("y_test_trans.csv", index=False)